First We import our required libraries

In [1]:

from google.colab import drive
#import os
!pip install pandas==1.5.3
#!pip install pandas==1.3.5
import pandas as pd

drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/TxtAnalytics_Proj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/gdrive
/content/gdrive/MyDrive/TxtAnalytics_Proj


In [ ]:
#SKIP
x1 = pd.read_pickle("post_split_Train_w_TFIDF_X_1.pkl")
x2 = pd.read_pickle("post_split_Train_w_TFIDF_X_2.pkl")
x3 = pd.read_pickle("post_split_Train_w_TFIDF_X_3.pkl")
x4 = pd.read_pickle("post_split_Train_w_TFIDF_X_4.pkl")
x5 = pd.read_pickle("post_split_Train_w_TFIDF_X_5.pkl")

data_raw_x = pd.concat([x1, x2, x3, x4, x5])

y1 = pd.read_pickle("post_split_Train_Y_1.pkl")
y2 = pd.read_pickle("post_split_Train_Y_2.pkl")
y3 = pd.read_pickle("post_split_Train_Y_3.pkl")
y4 = pd.read_pickle("post_split_Train_Y_4.pkl")
y5 = pd.read_pickle("post_split_Train_Y_5.pkl")
data_raw_y = pd.concat([y1,y2,y3,y4,y5])

data_raw = data_raw_x.merge(data_raw_y, how='inner', left_index=True, right_index=True)

print(data_raw.head())


In [ ]:
#SKIP
#data_raw["class"] = data_raw["class"].apply(lambda x: 1 if x =="suicide" else 0)
df_filtered = data_raw.loc[lambda x: x['class'] == 0]
df_filtered.head()

df_filtered.to_pickle("posts_after_lemmatized_non_suicidal.pkl")


Load Non Suicidal Text here

In [ ]:
#SKIP
df_filtered = pd.read_pickle("posts_after_lemmatized_non_suicidal.pkl")

!pip install fast_ml
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))
import fast_ml
from fast_ml.model_development import train_valid_test_split
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(df_filtered, target = 'class', 
                                                                            train_size=0.7, valid_size=0.2, test_size=0.1)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 KB 2.3 MB/s eta 0:00:00
The scikit-learn version is 1.2.1.


In [ ]:
#SKIP
X_train.to_pickle("X_Train_nonSuicidal.pkl")
X_valid.to_pickle("X_valid_nonSuicidal.pkl")
X_test.to_pickle("X_test_nonSuicidal.pkl")

y_train.to_pickle("y_Train_nonSuicidal.pkl")
y_valid.to_pickle("y_valid_nonSuicidal.pkl")
y_test.to_pickle("y_test_nonSuicidal.pkl")

Start Here

In [3]:
X_train = pd.read_pickle("X_Train_nonSuicidal.pkl")
#print(y_train.head())

In [4]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # min occurrences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=False,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

df_filtered_lemmatized = X_train["text_lemmatized"].tolist()

stop_list = ("suicidal", "suicide", "aah", "fuck")
df_filtered_lemmatized_removeSuicide = [[w for w in doc if w not in stop_list] for doc in df_filtered_lemmatized]
df_filtered_lemmatized_joined = [' '.join(x) for x in df_filtered_lemmatized_removeSuicide]  # joined to fit CountVectorizer
train1_vecs = vectorizer.fit_transform(df_filtered_lemmatized_joined)
feature_names = vectorizer.get_feature_names_out()
# display the first 20 tokens
feature_names[:20]

array(['aba', 'abandon', 'abandonment', 'abide', 'ability', 'able',
       'abnormal', 'abolish', 'abort', 'abortion', 'abroad', 'abs',
       'absence', 'absent', 'absolute', 'absolutely', 'absorb', 'absurd',
       'abuse', 'abused'], dtype=object)

In [ ]:
lda_non_suicidal_model = LatentDirichletAllocation(
             batch_size=128, 
             learning_decay=0.7,
             n_components=10,
             learning_method = 'online'
            )

lda_non_suicidal_output = lda_non_suicidal_model.fit_transform(train1_vecs)

print(lda_non_suicidal_model)  # Model attributes

LatentDirichletAllocation(learning_method='online')


In [ ]:
import joblib

# Save a model to disk
joblib.dump(lda_non_suicidal_model, 'non_suicidal_lda_model.jl')
joblib.dump(lda_non_suicidal_output, 'non_suicidal_lda_output.jl')

['non_suicidal_lda_output.jl']

In [5]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()
import joblib

lda_non_suicidal_model = joblib.load('non_suicidal_lda_model.jl')
lda_non_suicidal_output = joblib.load('non_suicidal_lda_output.jl')

print('The pyLDAvis-learn version is {}.'.format(pyLDAvis.__version__))
panel = pyLDAvis.lda_model.prepare(lda_non_suicidal_model, train1_vecs, vectorizer, mds='tsne')
panel
# Save the output|to the html file
#

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The pyLDAvis-learn version is 3.4.0.


/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
9      52.808430  43.383774       1        1  33.072903
8      54.037445 -60.080368       2        1  12.680800
7     -53.948231 -36.504036       3        1  12.056127
2      79.822617  -3.590449       4        1   9.922064
5      -5.611925  64.992607       5        1   7.817142
4     -11.075686 -19.222504       6        1   7.008445
6      -0.248071 -66.897224       7        1   5.348876
3      34.542149 -17.181519       8        1   5.056308
1     -43.799248  21.533064       9        1   4.214212
0       9.915132  21.299965      10        1   2.823124, topic_info=              Term          Freq         Total Category  logprob  loglift
2235        filler  28407.000000  28407.000000  Default  30.0000  30.0000
1478           day  11267.000000  11267.000000  Default  29.0000  29.0000
3423          like  25822.000000  25822.000000  Default  28.0000  28.0000
2752          help   6090.000000   6090.000000  Default  27.0000  27.0000
3917          need   6379.000000   6379.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
2912  hypocritical     28.225839     29.134360  Topic10  -7.1327   3.5356
3974        nipple     49.974539     72.062030  Topic10  -6.5615   3.2013
416          award     45.034889   1211.879641  Topic10  -6.6655   0.2748
331            ask     43.843936   5388.725509  Topic10  -6.6923  -1.2441
676           bore     36.626423   2364.510931  Topic10  -6.8722  -0.6002

[522 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
15        2  0.997464    absolutely
39        3  0.996219  accidentally
44        7  0.999552       account
72        8  0.999287           add
126       1  0.212122           ago
...     ...       ...           ...
6650      2  0.997937          yell
6656      1  0.375654           yes
6656      2  0.623495           yes
6668      7  0.999460       youtube
6686      1  0.997303          zoom

[770 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 8, 3, 6, 5, 7, 4, 2, 1])

In [6]:
pyLDAvis.save_html(panel, 'LDA_Nonsuicidal.html')